In [1]:
from ase import Atoms
import numpy as np

In [ ]:
class simple_md:
    
    # Input data should be an ASE atoms object for now. 
    def __init__(self, input_data):
        self.system = input_data
        self.n_atoms = len(input_data)
        self.temperature = 0
        self.positions = input_data.get_positions()
        self.velocities = input_data.get_velocities()
        self.forces = np.zeros.self.positions.shape()
        self.masses = input_data.get_masses()
        
        #initialize velocities
        #maxwell-boltzmann standard distribution (from TMP Chem)
        sigma_base = np.sqrt(2.0 * 1.98E-3 * temp / 3)
        for atom, vel in enumerate(velocities):
            sigma = sigma_base * masses[atom]**-0.5
            velocities[atom] = np.random.normal(0.0, sigma, 3)

        prev_positions = positions - (velocities*timestep)
                               
        